In [1]:
import os, sys, random, datetime, pickle, joblib, multiprocessing, gc, warnings, inspect, re, itertools

import numpy as np
import pandas as pd

from itertools import compress
from functools import reduce

DATA_PATH = '../../data/'

### Load Data

In [2]:
mastermapping = pd.read_excel(DATA_PATH + 'master_mapping_0918.xlsx')

In [3]:
mastermapping_sics = pd.read_csv(DATA_PATH + 'master_map_sics_filled.csv')

In [4]:
master_esg_calcs = pd.read_csv(DATA_PATH + 'master_esg_calcs_1024.csv')
master_esg_calcs = master_esg_calcs.rename(columns={'RIC_Primary':'RIC_primary'})

C:\Users\jinga\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (315,409,747,789,790,830,914,915,1047,1048,1057,1061,1065,1067,1069,1072,1076,1103,1176) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
esg_metrics_detail = pd.read_excel(DATA_PATH + 'esg_metrics_detail.xlsx')

C:\Users\jinga\anaconda3\envs\py37\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
sasb_mapdom = pd.read_excel(DATA_PATH + 'sasb_mapdom.xlsx')

In [7]:
portfolio = pd.read_excel( '../data/results_securities_checked/Securities_checked.xlsx')
# portfolio = portfolio[['Name','RIC','RIC_primary','ISIN','our_isin','Weight']]
# portfolio = portfolio.dropna(how='all',axis=0)

In [8]:
unmatched_portfolio = pd.read_csv( '../data/results_securities_checked/unmatched_calcs.csv').drop(columns='Unnamed: 0')

In [9]:
portfolio.RIC_primary.isna().sum()

20

In [10]:
portfolio.RIC_primary.notna().sum()

157

In [11]:
len(portfolio.loc[portfolio.RIC_primary.notna()].RIC_primary.unique().tolist())

156

In [12]:
import collections
a=pd.DataFrame.from_dict(dict(collections.Counter(portfolio.loc[portfolio.RIC_primary.notna()].RIC_primary.tolist())),orient='index').reset_index()

In [13]:
a.loc[a[0]==2]

,index,0
81,TCOM.OQ,2


### Data Info Mapping

In [14]:
to_merge_industry = mastermapping_sics[['RIC_primary','sics_industry_code']].drop_duplicates()

portfolio = portfolio.merge(to_merge_industry,on='RIC_primary',how='left')

portfolio = portfolio.rename(columns={'sics_industry_code':'industry'})

### task 2(b)

#### do we have at least 10 metrics for that company? Master_esg_calcs.csv, only look at ESG metrics, exclude XY Change, /.

#### 只看category_db.isin(['Environment','Social','Governance’], 然后排除名字里带 ‘Y Change’, ‘ / ’ 这样的就可以

In [15]:
#### get E, S, G metrics from category_DB
names_ESG_2b = esg_metrics_detail.loc[esg_metrics_detail['category_db'].isin(['Environmental','Social','Governance']),'calcs_name'].tolist()

#### remove Y Change names
names_ESG_2b = [x for x in names_ESG_2b if 'Y Change' not in x]

#### remove / names
names_ESG_2b = [x for x in names_ESG_2b if '/' not in x]

In [16]:
len(names_ESG_2b)

525

#### select the portfolio's RIC_primary among master_esg_calcs

In [17]:
master_esg_calcs_2b = master_esg_calcs[['RIC_primary','FY'] + names_ESG_2b]

master_esg_calcs_2b = master_esg_calcs_2b.loc[master_esg_calcs_2b['RIC_primary'].isin(portfolio['RIC_primary'])]

master_esg_calcs_2b = master_esg_calcs_2b.reset_index(drop=True)

In [18]:
coverage_2b = master_esg_calcs_2b.groupby(['RIC_primary','FY']).apply(lambda x: x[names_ESG_2b].notna().sum().sum())
coverage_2b = coverage_2b.reset_index().rename(columns={0:'valid_data_points_num'})

In [19]:
coverage_2b['valid_data_points_coverage'] = coverage_2b['valid_data_points_num']/len(names_ESG_2b)

In [20]:
amendment_2b = unmatched_portfolio[['RIC','FY'] + names_ESG_2b].groupby(['RIC','FY']).apply(lambda x: x[names_ESG_2b].notna().sum().sum()).reset_index().rename(columns={0:'valid_data_points_num'})
amendment_2b['valid_data_points_coverage'] = amendment_2b['valid_data_points_num']/len(names_ESG_2b)

### task 2(c)

#### 要看每个公司对应的industry, 在sasb_mapdom里面有多少个material的trcode, 然后你通过esg_metrics_detail里 trcode和 index_metric=1的可以找到在calc_name. 这些对上的, 是当前这家公司的所有的material的metrics, 然后你再数这些metrics里面, 多少非空

#### 做2c的时候不需要category_db.isin(['Environment','Social','Governance’] 还有  ‘Y Change’ 这个筛选哈, 通过material的mapping 本身就排除了

#### Step 1 ：industry - trcode mapping data

In [21]:
to_merge_trcode = sasb_mapdom[['industry','trcode']].dropna().drop_duplicates()

to_merge_trcode

,industry,trcode
0,RR-PP,TR.AnalyticRenewEnergyUse
1,RR-ST,TR.EcoDesignProducts
2,CG-AM,TR.PercentageGreenProducts
3,CG-BF,TR.PercentageGreenProducts
4,CG-EC,TR.AsianMinoritiesEmployeesPercentage
...,...,...
1138,TR-AP,TR.WasteTotal
1139,TR-RA,TR.EmployeeFatalities
1140,TR-RA,TR.ContractorFatalities
1141,TR-RO,TR.EmployeeFatalities


#### Step 2: 根据trcode，将index_metric merge至上一步结果

In [22]:
esg_metrics_detail_2c = esg_metrics_detail[['trcode','index_metric','calcs_name']].dropna()
esg_metrics_detail_2c['trcode_lower'] = [x.split('(')[0].lower() for x in esg_metrics_detail_2c['trcode']]
esg_metrics_detail_2c

,trcode,index_metric,calcs_name,trcode_lower
0,TR.CO2EmissionTotal,1.0,Total CO2 Emissions,tr.co2emissiontotal
4,TR.AnalyticCO2,1.0,Total CO2 Emissions / Revenue,tr.analyticco2
20,TR.CO2DirectScope1,1.0,"CO2 Emissions, Scope 1 / Revenue",tr.co2directscope1
24,TR.CO2IndirectScope2,1.0,"CO2 Emissions, Scope 2 / Revenue",tr.co2indirectscope2
28,TR.CO2IndirectScope3,1.0,"CO2 Emissions, Scope 3 / Revenue",tr.co2indirectscope3
...,...,...,...,...
905,TR.BlackorAfricanAmericanMinoritiesManagersPer...,1.0,Black or African American - Minorities Manager...,tr.blackorafricanamericanminoritiesmanagersper...
906,TR.HispanicorLatinoMinoritiesManagersPercentage,1.0,Hispanic or Latino - Minorities Managers Perce...,tr.hispanicorlatinominoritiesmanagerspercentage
907,TR.WhiteMinoritiesManagersPercentage,1.0,White - Minorities Managers Percentage,tr.whiteminoritiesmanagerspercentage
908,TR.EnvProducts,1.0,Environmental Products,tr.envproducts


In [23]:
to_merge_trcode['trcode_lower'] = [x.lower() for x in to_merge_trcode['trcode']]
to_merge_trcode

,industry,trcode,trcode_lower
0,RR-PP,TR.AnalyticRenewEnergyUse,tr.analyticrenewenergyuse
1,RR-ST,TR.EcoDesignProducts,tr.ecodesignproducts
2,CG-AM,TR.PercentageGreenProducts,tr.percentagegreenproducts
3,CG-BF,TR.PercentageGreenProducts,tr.percentagegreenproducts
4,CG-EC,TR.AsianMinoritiesEmployeesPercentage,tr.asianminoritiesemployeespercentage
...,...,...,...
1138,TR-AP,TR.WasteTotal,tr.wastetotal
1139,TR-RA,TR.EmployeeFatalities,tr.employeefatalities
1140,TR-RA,TR.ContractorFatalities,tr.contractorfatalities
1141,TR-RO,TR.EmployeeFatalities,tr.employeefatalities


In [24]:
to_merge_trcode = to_merge_trcode.merge(esg_metrics_detail_2c[['trcode_lower','index_metric','calcs_name']].dropna(),on='trcode_lower',how='left')
to_merge_trcode = to_merge_trcode.loc[to_merge_trcode['index_metric']==1]
to_merge_trcode

,industry,trcode,trcode_lower,index_metric,calcs_name
0,RR-PP,TR.AnalyticRenewEnergyUse,tr.analyticrenewenergyuse,1.0,Renewable Energy / Total Energy
1,RR-ST,TR.EcoDesignProducts,tr.ecodesignproducts,1.0,Eco-Design Products
2,CG-AM,TR.PercentageGreenProducts,tr.percentagegreenproducts,1.0,Percentage of Green Products
3,CG-BF,TR.PercentageGreenProducts,tr.percentagegreenproducts,1.0,Percentage of Green Products
4,CG-EC,TR.AsianMinoritiesEmployeesPercentage,tr.asianminoritiesemployeespercentage,1.0,Asian - Minorities Employees Percentage
...,...,...,...,...,...
775,TR-AP,TR.WasteTotal,tr.wastetotal,1.0,Total Waste / Revenue
776,TR-RA,TR.EmployeeFatalities,tr.employeefatalities,1.0,Employee Fatalities / Total Employees
777,TR-RA,TR.ContractorFatalities,tr.contractorfatalities,1.0,Contractor Fatalities / Total Employees
778,TR-RO,TR.EmployeeFatalities,tr.employeefatalities,1.0,Employee Fatalities / Total Employees


In [25]:
to_merge_trcode.index_metric.isna().sum()

0

#### Step 3: 每个公司对应的calcs_name的valid coverage

In [26]:
master_esg_calcs_2c = master_esg_calcs.loc[master_esg_calcs['RIC_primary'].isin(portfolio['RIC_primary'])]
master_esg_calcs_2c = master_esg_calcs_2c.reset_index(drop=True)
master_esg_calcs_2c

,RIC_primary,RIC,Date,FY,Forward PE,Forward PE | 1Y Change,Forward PE | 3Y Change,Forward PE | 5Y Change,Gross Margin,Gross Margin | 1Y Change,...,Debt to Equity,Debt to Equity | 1Y Change,Debt to Equity | 3Y Change,Debt to Equity | 5Y Change,Enterprise Value - Actual,Cash and Equivalents,Date_temp,ret_1yr,ret_3yr,ret_5yr
0,0173.HK,0173.HK,2014-12-31 00:00:00+00:00,FY2014,12.043956,NaN,NaN,NaN,58.85716,NaN,...,70.678355,NaN,NaN,NaN,NaN,NaN,201412.0,-0.099819,1.326353,0.684649
1,0173.HK,0173.HK,2015-12-31 00:00:00+00:00,FY2015,9.744712,-2.299244,NaN,NaN,64.41361,5.55645,...,57.661005,-0.184177,NaN,NaN,NaN,NaN,201512.0,-0.156006,-0.004584,0.350751
2,0173.HK,0173.HK,2016-12-31 00:00:00+00:00,FY2016,5.089562,-4.655150,NaN,NaN,47.02159,-17.39202,...,43.425684,-0.246880,NaN,NaN,NaN,NaN,201612.0,0.126923,-0.143824,1.212632
3,0173.HK,0173.HK,2017-12-31 00:00:00+00:00,FY2017,4.715627,-0.373935,-7.328329,NaN,50.11894,3.09735,...,52.792720,0.215703,-0.253057,NaN,NaN,NaN,201712.0,0.229478,0.169377,0.379177
4,0173.HK,0173.HK,2018-12-31 00:00:00+00:00,FY2018,3.490762,-1.224865,-6.253950,NaN,44.88942,-5.22952,...,52.498561,-0.005572,-0.089531,NaN,NaN,NaN,201812.0,-0.092228,0.257744,-0.044433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,ZZZ.TO,ZZZ.TO,2018-12-31 00:00:00+00:00,FY2018,11.435934,-10.089261,-4.424231,NaN,30.42568,0.55353,...,56.205884,0.392867,0.050772,NaN,674.306547,NaN,201812.0,-0.387083,0.239195,0.332091
1400,ZZZ.TO,ZZZ.TO,2019-12-31 00:00:00+00:00,FY2019,13.080906,1.644972,-8.477146,NaN,31.34458,0.91890,...,156.952309,1.792453,2.309013,-0.642911,854.751943,NaN,201912.0,0.051150,-0.237828,0.400228
1401,ZZZ.TO,ZZZ.TO,2020-12-31 00:00:00+00:00,FY2020,15.330086,2.249180,-6.195109,-0.530079,30.49667,-0.84791,...,106.332313,-0.322518,1.635075,0.987889,1040.614361,NaN,202012.0,0.332886,-0.141265,0.736190
1402,ZZZ.TO,ZZZ.TO,2021-12-31 00:00:00+00:00,FY2021,15.054997,-0.275089,3.619063,-6.503055,34.45447,3.95780,...,95.041144,-0.106188,0.690947,1.003745,1370.814856,NaN,202112.0,0.453373,1.036267,0.476464


In [27]:
coverage_2c = []
for i in master_esg_calcs_2c['RIC_primary'].unique():
# i = '002352.SZ'
    industry_tmp = portfolio.loc[portfolio['RIC_primary']==i,'industry'].values[0]
    
    columns_tmp = to_merge_trcode.loc[to_merge_trcode['industry']==industry_tmp]['calcs_name'].tolist()
    
    df_tmp = master_esg_calcs_2c.loc[master_esg_calcs_2c['RIC_primary']==i][['RIC_primary','FY']+columns_tmp]
    
    groupby_tmp = df_tmp.groupby(['RIC_primary','FY']).apply(lambda x: pd.DataFrame([x[columns_tmp].notna().sum().sum(),
                                                                        len(columns_tmp),
                                                                        x[columns_tmp].notna().sum().sum()/len(columns_tmp)]).T)
    
    coverage_2c.append(groupby_tmp)

In [28]:
coverage_2c = pd.concat(coverage_2c).reset_index()
coverage_2c = coverage_2c.rename(columns={0:'valid_materials_num',
                                          1:'total_materials_num',
                                          2:'valid_materials_coverage'})
coverage_2c = coverage_2c.drop(columns='level_2')

### task 2(d)

#### 单独导出下co2 emissions的四列 (Total Co2 emissions, co2 emissions scope 1, 2, 3), 然后分别算下覆盖率

In [29]:
emis_cols = ['Total CO2 Emissions','CO2 Emissions, Scope 1','CO2 Emissions, Scope 2','CO2 Emissions, Scope 3']

In [30]:
master_esg_calcs_2d = master_esg_calcs[['RIC_primary','FY'] + emis_cols]

master_esg_calcs_2d = master_esg_calcs_2d.loc[master_esg_calcs_2d['RIC_primary'].isin(portfolio['RIC_primary'])]

master_esg_calcs_2d = master_esg_calcs_2d.reset_index(drop=True)

In [31]:
coverage_2d = master_esg_calcs_2d.groupby(['RIC_primary','FY']).apply(lambda x: x[emis_cols].notna().sum().sum())
coverage_2d = coverage_2d.reset_index().rename(columns={0:'emissions_valid_data_points_num'})

coverage_2d['emissions_valid_data_points_coverage'] = coverage_2d['emissions_valid_data_points_num']/len(emis_cols)

In [32]:
amendment_2d = unmatched_portfolio[['RIC','FY'] + emis_cols].groupby(['RIC','FY']).apply(lambda x: x[emis_cols].notna().sum().sum()).reset_index().rename(columns={0:'emissions_valid_data_points_num'})
amendment_2d['emissions_valid_data_points_coverage'] = amendment_2d['emissions_valid_data_points_num']/len(emis_cols)

### 合并

In [48]:
final_output = coverage_2b.merge(coverage_2c,on=['RIC_primary','FY'],how='left').merge(coverage_2d,on=['RIC_primary','FY'],how='left')

In [49]:
final_amendment = amendment_2b.merge(amendment_2d,on=['RIC','FY'],how='left')

In [50]:
final_output = final_output.merge(portfolio[['RIC_primary','CURR','TICKER','DESCRIPTION','CUSIP','ISIN']],
                   on='RIC_primary',how='left')

In [51]:
final_amendment = final_amendment.merge(portfolio[['RIC_searched','CURR','TICKER','DESCRIPTION','CUSIP','ISIN']].rename(columns={'RIC_searched':'RIC'}),
                   on='RIC',how='left')

In [52]:
final_output['RIC'] = np.nan
final_amendment['RIC_primary'] = np.nan
final_amendment['valid_materials_num'],final_amendment['total_materials_num'],final_amendment['valid_materials_coverage'] = np.nan,np.nan,np.nan

In [53]:
to_save_cols = ['RIC','RIC_primary', 'FY', 'valid_data_points_num',
       'valid_data_points_coverage', 'valid_materials_num',
       'total_materials_num', 'valid_materials_coverage',
       'emissions_valid_data_points_num',
       'emissions_valid_data_points_coverage', 'CURR', 'TICKER', 'DESCRIPTION',
       'CUSIP', 'ISIN']
final_output = final_output[to_save_cols]
final_amendment = final_amendment[to_save_cols]

In [54]:
to_save = pd.concat([final_output,final_amendment]).reset_index(drop=True)
to_save.ISIN.isna().sum()

0

In [41]:
to_save.to_excel('../data/results_securities_checked/results/Securities_checked_metrics_coverage.xlsx',index=False)